# FINANCIAL INFORMATION SYSTEMS PROJECT PART B


This block of code specifies the parameter choices for the Moving Average (MA) Crossover and Bollinger Bands (BB) strategies, as well as the final list of strategies to be used in the portfolio.

## Question 2: Get the Previous Windows for further Analysis

In [1]:
# Choices - Question 5
maFast_flat = 105
maSlow_flat = 255

maFast_short = 105
maSlow_short = 255

bbWindow = 26
bbStdevBand = 3

myStrategies = ['AAPL-MAFlat', 'AMZN-MAFlat', 'SPY-MAFlat', 'AUD-BB', 'FBNDX-BMK-MA', 'GE-MAShort', 'GOLD-MAShort', 'INTC-BB']

In [2]:
#Enhanced pretty print function to only format part of the dataframe
def print_pct_data(data_frame, format_spec='{:.4%}', na_rep='NaN', index_slice=None):
    return data_frame.style.format(format_spec, na_rep=na_rep, subset=index_slice)

In [3]:
# Define the function to calculate performance statistics (it drops the NA row and also takes the cumulative sum for drawdown calculation)
def calcPerformanceStatistics(data_frame, annualization_factor=260, printScreen=True, percent_format_string='{:.2%}'):
    data_frame = data_frame.dropna()
    data_frame_cumsum = data_frame.cumsum()

    #Calculate the performance statistics into a single data frame
    summary_stats = pd.DataFrame([annualization_factor*data_frame.mean(),
                                  np.sqrt(annualization_factor)*data_frame.std(),
                                 data_frame.mean() / data_frame.std() * np.sqrt(annualization_factor),
                                  (data_frame_cumsum.fillna(0) - np.maximum.accumulate(data_frame_cumsum.fillna(0))).min(axis=0),
                                  (data_frame.dropna() >= 0).sum() / data_frame.dropna().shape[0]],
                                 index=['Annual Return', 'Annualized Risk', 'Sharpe Ratio', 'Max Drawdown', 'Success Ratio'])

    index_slice = pd.IndexSlice[['Annual Return', 'Annualized Risk', 'Max Drawdown', 'Success Ratio'], :]

    # Print the summary statistics if printScreen is True
    #if printScreen:
        #display(print_pct_data(summary_stats, index_slice=index_slice)) #ChatGPT was used to debug this code and add the display() function since the data frame was not printing otherwise.

    # Return the summary statistics DataFrame
    return summary_stats

This block of code is used to set up the environment by importing essential libraries for data analysis (pandas, numpy) and visualization (matplotlib). Pandas is used for data manipulation, numpy for numerical operations, and matplotlib for creating plots and handling tick formatting.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

## Reference

In [5]:
# Function to calculate MA crossover strategies
def movingAverageCrossover(data, fastWindow, slowWindow):
    df = data.copy()
    # Check if the DataFrame has more than one column
    if df.shape[1] != 1:
        raise ValueError("Input DataFrame must have exactly one column")
    # Get the instrument name
    instrumentName = df.columns[0]
    # Calculate fast and slow SMAs
    df['FastSMA'] = df[instrumentName].rolling(fastWindow).mean()
    df['SlowSMA'] = df[instrumentName].rolling(slowWindow).mean()
    # Drop null values
    df.dropna(inplace=True)
    # Calculate log returns
    df['BMKReturns'] = np.log(df[instrumentName] / df[instrumentName].shift(1))
    # SMA strategy: if rising trend, stay long, otherwise go flat
    df['PositionFlat'] = np.where(df['FastSMA'] > df['SlowSMA'], 1, 0)
    # SMA strategy: if rising trend, stay long, otherwise go short
    df['PositionShort'] = np.where(df['FastSMA'] > df['SlowSMA'], 1, -1)
    # Calculate the strategy returns, taking care to push down the Position column
    df['MAFlatReturns'] = df['PositionFlat'].shift(1) * df['BMKReturns']
    df['MAShortReturns'] = df['PositionShort'].shift(1) * df['BMKReturns']
    # Create the result DataFrame
    result = pd.DataFrame({
        f'{instrumentName}-BMK-MA': df['BMKReturns'],
        f'{instrumentName}-MAFlat': df['MAFlatReturns'],
        f'{instrumentName}-MAShort': df['MAShortReturns']
    })
    return result

In [6]:
# Function to calculate bollinger bands
def bollingerBands(dataFrame, lookbackWindow, stdevBand):
    if len(dataFrame.columns) > 1:
      raise ValueError("The input DataFrame must have only one column of prices.")

    data_frame = dataFrame.copy()
    instrument = data_frame.columns[0]
    data_frame.loc[:,'Mean'] = data_frame[instrument].rolling(lookbackWindow).mean()
    data_frame.loc[:,'Stdev'] = data_frame[instrument].rolling(lookbackWindow).std()
    data_frame.dropna(inplace=True)
    data_frame.loc[:,'Upper'] = data_frame['Mean'] + stdevBand * data_frame['Stdev']
    data_frame.loc[:,'Lower'] = data_frame['Mean'] - stdevBand * data_frame['Stdev']

    date0 = data_frame.index[0]
    if data_frame.loc[date0,instrument] >= data_frame.loc[date0, 'Upper']:
        data_frame.loc[date0, 'Position'] = -1
    elif data_frame.loc[date0,instrument] <= data_frame.loc[date0,'Lower']:
        data_frame.loc[date0, 'Position'] = 1
    else:
        data_frame.loc[date0, 'Position'] = 0

    # then loop over the entire data set
    for i in range(1, data_frame.shape[0]):
        today=data_frame.index[i]
        yesterday=data_frame.index[i-1]
        if data_frame.loc[today,instrument] >= data_frame.loc[today,'Upper']: # if close is above upper
            data_frame.loc[today,'Position'] = -1 # then go short
        elif data_frame.loc[today,instrument] <= data_frame.loc[today,'Lower']: # if close is below lower
            data_frame.loc[today,'Position'] = 1 # then go long
        elif data_frame.loc[yesterday,'Position'] == -1 and data_frame.loc[today,instrument] <= data_frame.loc[today,'Mean']: # if prev day is short and we're now below the mean
            data_frame.loc[today,'Position']=0 # then flatten
        elif data_frame.loc[yesterday,'Position'] == 1 and data_frame.loc[today, instrument] >= data_frame.loc[today, 'Mean']: # conversely...
            data_frame.loc[today,'Position']=0 # then also flatten
        else: # otherwise just hold yesterday's position
            data_frame.loc[today,'Position']=data_frame.loc[yesterday,'Position']

    data_frame.loc[:,'BMKReturns'] = np.log(data_frame[instrument] / data_frame[instrument].shift(1))
    data_frame.loc[:,'BBReturns'] = data_frame['Position'].shift(1) * data_frame['BMKReturns']
    data_frame = data_frame[['BMKReturns', 'BBReturns']]

    data_frame = data_frame.rename(columns={'BMKReturns': f'{instrument}-BMK-BB', 'BBReturns': f'{instrument}-BB'})
    return data_frame

In [7]:
# Function which will run the 2 technical trading strategy functions
def runMovingAverageAndBB(data_frame, fastWindow, slowWindow, bbWindow, stdevBand):
    # Run moving average crossover strategy
    ma_results = movingAverageCrossover(data_frame, fastWindow, slowWindow)
    
    # Run Bollinger Bands strategy
    bb_results = bollingerBands(data_frame, bbWindow, stdevBand)
    
    # Merge the two DataFrames on the date index (inner join)
    combined_results = ma_results.join(bb_results, how='inner')
    
    # Check if BMK-MA and BMK-BB columns are identical (as per the requirement)
    #if not combined_results[f'{data_frame.columns[0]}-BMK-MA'].equals(combined_results[f'{data_frame.columns[0]}-BMK-BB']):
        #raise ValueError("BMK-MA and BMK-BB columns are not identical. Check your logic.")
    
    return combined_results

In [8]:
# Function to read the CSV file and print its head, tail, null values & summary statistics.
def read_price_data(filename, index_col=0, parse_dates=True):
    df = pd.read_csv(filename, index_col=index_col, parse_dates=parse_dates)
    print(df.head())
    print(df.tail())
    print(df.isnull().sum())
    print(df.describe())
    df
    return df

In [9]:
# Define the selected instruments
selected_instruments = ['AAPL', 'AMZN', 'SPY', 'GE', 'INTC', 'FBNDX', 'GOLD', 'AUD']

In [10]:
# List of instruments and corresponding strategies
instruments = ['AAPL', 'AMZN', 'ATT', 'GE', 'INTC', 'SPY', 'FBNDX', 'LEHM', 'GOLD', 'SILVER', 'AUD', 'EUR']

# Chosen strategies and parameters for each instrument
strategy_params = {
    'AAPL': ('MAFlat', maFast_flat, maSlow_flat, bbWindow, bbStdevBand),
    'AMZN': ('MAFlat', maFast_flat, maSlow_flat, bbWindow, bbStdevBand),
    'ATT': ('MAShort', maFast_short, maSlow_short, bbWindow, bbStdevBand),
    'GE': ('MAShort', maFast_short, maSlow_short, bbWindow, bbStdevBand),
    'INTC': ('BB', maFast_short, maSlow_short, bbWindow, bbStdevBand),
    'SPY': ('MAFlat', maFast_flat, maSlow_flat, bbWindow, bbStdevBand),
    'FBNDX': ('BMK-MA', maFast_flat, maSlow_flat, bbWindow, bbStdevBand),
    'LEHM': ('BMK-MA', maFast_flat, maSlow_flat, bbWindow, bbStdevBand),
    'GOLD': ('MAShort', maFast_short, maSlow_short, bbWindow, bbStdevBand),
    'SILVER': ('BB', maFast_short, maSlow_short, bbWindow, bbStdevBand),
    'AUD': ('BB', maFast_short, maSlow_short, bbWindow, bbStdevBand),
    'EUR': ('BB', maFast_short, maSlow_short, bbWindow, bbStdevBand)
}

In [11]:
# Function to calculate performance statistics
def calculate_performance_stats(df):
    performance_stats = {}
    performance_stats['Mean Return'] = df.mean() * 260  # Annualized mean returns
    performance_stats['Volatility'] = df.std() * np.sqrt(260)  # Annualized volatility
    performance_stats['Sharpe Ratio'] = performance_stats['Mean Return'] / performance_stats['Volatility']
    return pd.DataFrame(performance_stats)

In [12]:
def calculate_portfolio_sharpe(portfolio_returns):
    # Calculate annualized return and volatility
    annualized_return = portfolio_returns.mean() * 260  # Assuming 260 trading days in a year
    annualized_volatility = portfolio_returns.std() * np.sqrt(260)  # Annualized volatility
    sharpe_ratio = annualized_return / annualized_volatility  # Sharpe ratio
    return sharpe_ratio

### Question 1: Load the pricesB DataFrame 

In [13]:
# Load the dataset PricesProjectA.csv
pricesB_df = read_price_data('PricesProjectB.csv')

                AAPL      AMZN        ATT     AUD     EUR  FBNDX          GE  \
1999-12-31  0.917969  3.806250  36.820244  0.6567  1.0062   6.89  396.794861   
2000-01-03  0.999442  4.468750  35.498489  0.6585  1.0243   6.85  384.615387   
2000-01-04  0.915179  4.096875  33.421452  0.6550  1.0296   6.87  369.230774   
2000-01-05  0.928571  3.487500  33.940708  0.6579  1.0321   6.84  368.589752   
2000-01-06  0.848214  3.278125  33.043808  0.6533  1.0328   6.86  373.517639   

              GOLD      INTC        LEHM  SILVER       SPY  
1999-12-31  288.00  41.15625  798.154631   5.380  146.8750  
2000-01-03  289.00  43.50000  793.868670   5.405  145.4375  
2000-01-04  282.25  41.46875  796.761677   5.325  139.7500  
2000-01-05  280.00  41.81250  793.118215   5.145  140.0000  
2000-01-06  281.10  39.37500  795.091984   5.120  137.7500  
                  AAPL        AMZN        ATT     AUD     EUR  FBNDX  \
2022-10-25  152.339996  120.599998  17.690001  0.6394  0.9966   6.88   
2022-10-2

### Question 3: Re-run Your Part A Chosen Portfolio for the Full Period (1999-12-31 to 2022-10-31)

#### Reasoning:
In this question, we re-ran our Part A chosen portfolio, applying the same technical strategies (MAFlat, MAShort, and Bollinger Bands) to the chosen instruments. This time, we extended the time period to include data through **October 31, 2022**. The goal was to evaluate the performance of our strategies over a longer time frame and observe whether they continue to provide favorable risk-adjusted returns.

The technical strategy functions (`movingAverageCrossover`, `bollingerBands`, and `runMovingAverageAndBB`) were re-used from Part A, ensuring consistency in strategy application. We created a new dataFrame that contains the daily returns of the 8 chosen strategies, formatted similarly to the one we created in Part A (Question 8). 

In [14]:
# Create an empty list to store individual DataFrames
dataframesB = []
dataframesB_inSample = []
dataframesB_outSample = []

In [15]:
# Run the function on all 12 instruments and store the results
for instrument, params in strategy_params.items():
    strategy_type, fastWindow, slowWindow, bbWindow, stdevBand = params
    # Use the previously defined runMovingAverageAndBB function
    resultB_df = runMovingAverageAndBB(pricesB_df[[instrument]], fastWindow, slowWindow, bbWindow, stdevBand)
    
    # Append the result to the list of dataframes
    dataframesB.append(resultB_df)

In [16]:
# Concatenate all 12 dataFrames horizontally to create a master DataFrame with 60 columns
masterB_df = pd.concat(dataframesB, axis=1)

# Display the master DataFrame
#masterB_df.head()

In [17]:
# Sort the strategies alphabetically
myStrategies_sorted = sorted(myStrategies)

In [18]:
masterB_df.head().dropna()

,AAPL-BMK-MA,AAPL-MAFlat,AAPL-MAShort,AAPL-BMK-BB,AAPL-BB,AMZN-BMK-MA,AMZN-MAFlat,AMZN-MAShort,AMZN-BMK-BB,AMZN-BB,...,AUD-BMK-MA,AUD-MAFlat,AUD-MAShort,AUD-BMK-BB,AUD-BB,EUR-BMK-MA,EUR-MAFlat,EUR-MAShort,EUR-BMK-BB,EUR-BB
2001-01-05,-0.041128,-0.0,0.041128,-0.041128,-0.0,-0.062390,-0.0,0.062390,-0.062390,-0.0,...,0.009840,0.0,-0.009840,0.009840,0.0,0.007868,0.0,-0.007868,0.007868,0.0
2001-01-08,0.011385,0.0,-0.011385,0.011385,0.0,0.025425,0.0,-0.025425,0.025425,0.0,...,-0.007900,-0.0,0.007900,-0.007900,-0.0,-0.010821,-0.0,0.010821,-0.010821,-0.0
2001-01-09,0.037042,0.0,-0.037042,0.037042,0.0,0.091881,0.0,-0.091881,0.091881,0.0,...,-0.006897,-0.0,0.006897,-0.006897,-0.0,-0.003492,-0.0,0.003492,-0.003492,-0.0
2001-01-10,-0.037042,-0.0,0.037042,-0.037042,-0.0,0.007605,0.0,-0.007605,0.007605,0.0,...,-0.017544,-0.0,0.017544,-0.017544,-0.0,-0.006807,-0.0,0.006807,-0.006807,-0.0


In [19]:
# Subset the master_df to include only the columns for the chosen strategies
myStrategies_dfB = masterB_df[myStrategies_sorted]

In [20]:
# Display the first few rows of the resulting DataFrame
myStrategies_dfB

,AAPL-MAFlat,AMZN-MAFlat,AUD-BB,FBNDX-BMK-MA,GE-MAShort,GOLD-MAShort,INTC-BB,SPY-MAFlat
2001-01-04,NaN,NaN,0.0,NaN,NaN,NaN,-0.0,NaN
2001-01-05,-0.0,-0.0,0.0,0.002770,-0.015728,-0.002240,-0.0,-0.0
2001-01-08,0.0,0.0,-0.0,0.001382,-0.037690,-0.000746,-0.0,0.0
2001-01-09,0.0,0.0,-0.0,-0.002766,-0.020791,0.002238,0.0,-0.0
2001-01-10,-0.0,0.0,-0.0,-0.002774,0.001400,0.010513,0.0,0.0
...,...,...,...,...,...,...,...,...
2022-10-25,0.0,0.0,0.0,0.010227,0.004919,-0.002053,0.0,0.0
2022-10-26,-0.0,-0.0,0.0,0.004351,-0.033143,-0.006872,-0.0,-0.0
2022-10-27,-0.0,-0.0,-0.0,0.004332,-0.007131,0.000757,-0.0,-0.0
2022-10-28,0.0,-0.0,-0.0,-0.001442,-0.030197,0.011154,0.0,0.0


In [21]:
myStrategies_dfB.dropna()

,AAPL-MAFlat,AMZN-MAFlat,AUD-BB,FBNDX-BMK-MA,GE-MAShort,GOLD-MAShort,INTC-BB,SPY-MAFlat
2001-01-05,-0.0,-0.0,0.0,0.002770,-0.015728,-0.002240,-0.0,-0.0
2001-01-08,0.0,0.0,-0.0,0.001382,-0.037690,-0.000746,-0.0,0.0
2001-01-09,0.0,0.0,-0.0,-0.002766,-0.020791,0.002238,0.0,-0.0
2001-01-10,-0.0,0.0,-0.0,-0.002774,0.001400,0.010513,0.0,0.0
2001-01-11,0.0,0.0,0.0,-0.001390,0.041102,0.002646,0.0,0.0
...,...,...,...,...,...,...,...,...
2022-10-25,0.0,0.0,0.0,0.010227,0.004919,-0.002053,0.0,0.0
2022-10-26,-0.0,-0.0,0.0,0.004351,-0.033143,-0.006872,-0.0,-0.0
2022-10-27,-0.0,-0.0,-0.0,0.004332,-0.007131,0.000757,-0.0,-0.0
2022-10-28,0.0,-0.0,-0.0,-0.001442,-0.030197,0.011154,0.0,0.0


#### Key Insights:
1. **In-Sample Period**: For consistency, the strategies were re-applied starting from **12/31/1999** up until **12/31/2018**, as done in Part A.
2. **Out-of-Sample Period**: The out-of-sample analysis was extended to cover the period from **01/02/2019 to 10/31/2022**.
3. **Extended Time Frame**: The inclusion of additional data, especially through market fluctuations like the COVID-19 pandemic, provided insights into the robustness and adaptability of our technical strategies over time.

#### Results:
The extended period analysis helps us observe how well our strategies perform not only during the initial in-sample period but also during more recent market events. This provides a broader view of the effectiveness of the strategies across various market conditions, helping us understand potential overfitting or strategy limitations over time.

### Question 4: Apply MVO Maximum-Sharpe Weights and Calculate Performance for In-Sample and Out-of-Sample Periods

#### Reasoning:
In this question, we applied the same Maximum Sharpe weights obtained from Part A (Question 14) using the Mean-Variance Optimization (MVO) method. We added a 9th column to our dataFrame, representing the MVO-weighted Maximum Sharpe portfolio. This step mirrored what we did in Part A (Question 16) but with the extended data period from **12/31/1999 to 10/31/2022**.

The key goal was to compare the performance of this portfolio across two distinct periods:
1. **In-Sample Period**: Ending on **12/31/2018**, identical to the period used in Part A.
2. **Out-of-Sample Period**: Spanning from **01/02/2019 to 10/31/2022**, providing an extended evaluation of the portfolio's performance after the initial training period.

In [22]:
inSample = pd.IndexSlice[:'2018-12-31',:]
outOfSample = pd.IndexSlice['2019-01-02':'2022-10-31',:]

# Découpage du DataFrame
pricesB_df_inSample = pricesB_df.loc[inSample]
pricesB_df_outOfSample = pricesB_df.loc[outOfSample]

In [23]:
def remove_duplicate_columns(df):
    # Create a dictionary to store unique columns
    unique_cols = {}
    
    for col in df.columns:
        # Convert the column to a string for comparison
        col_contents = df[col].values.tobytes()
        
        # If the column content hasn't been seen, add it to the dictionary
        if col_contents not in unique_cols:
            unique_cols[col_contents] = col
        # Otherwise, rename the duplicate column
        else:
            df = df.rename(columns={col: unique_cols[col_contents]})
    
    # Remove duplicate columns
    df = df.loc[:, ~df.columns.duplicated()]
    
    return df

# Using the function
myStrategies_dfB_cleaned = remove_duplicate_columns(myStrategies_dfB)

myStrategies_dfB_cleaned 

,AAPL-MAFlat,AMZN-MAFlat,AUD-BB,FBNDX-BMK-MA,GE-MAShort,GOLD-MAShort,INTC-BB,SPY-MAFlat
2001-01-04,NaN,NaN,0.0,NaN,NaN,NaN,-0.0,NaN
2001-01-05,-0.0,-0.0,0.0,0.002770,-0.015728,-0.002240,-0.0,-0.0
2001-01-08,0.0,0.0,-0.0,0.001382,-0.037690,-0.000746,-0.0,0.0
2001-01-09,0.0,0.0,-0.0,-0.002766,-0.020791,0.002238,0.0,-0.0
2001-01-10,-0.0,0.0,-0.0,-0.002774,0.001400,0.010513,0.0,0.0
...,...,...,...,...,...,...,...,...
2022-10-25,0.0,0.0,0.0,0.010227,0.004919,-0.002053,0.0,0.0
2022-10-26,-0.0,-0.0,0.0,0.004351,-0.033143,-0.006872,-0.0,-0.0
2022-10-27,-0.0,-0.0,-0.0,0.004332,-0.007131,0.000757,-0.0,-0.0
2022-10-28,0.0,-0.0,-0.0,-0.001442,-0.030197,0.011154,0.0,0.0


In [24]:
np.random.seed(64)  # For reproducibility

# returns = myStrategies_df.pct_change().dropna()  # Daily returns of the instruments
annual_returns = myStrategies_dfB.mean() * 260  # Annualized returns
cov_matrix = myStrategies_dfB.cov() * 260  # Annualized covariance matrix

num_portfolios = 5000  # Monte Carlo simulation
resultsB = np.zeros((3, num_portfolios))
weights_recordB = []

for i in range(num_portfolios):
    weights = np.random.random(len(annual_returns))
    weights /= np.sum(weights)  # Normalize weights to sum to 1
    weights_recordB.append(weights)

    # Portfolio return and volatility
    portfolio_return = np.sum(weights * annual_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Sharpe ratio assuming risk-free rate = 0
    sharpe_ratio = portfolio_return / portfolio_volatility

    resultsB[0, i] = portfolio_return
    resultsB[1, i] = portfolio_volatility
    resultsB[2, i] = sharpe_ratio

weights_recordB = pd.DataFrame(weights_recordB, columns=myStrategies_dfB.columns)

In [25]:
max_sharpe_idx = np.argmax(resultsB[2, :])
weights_max_sharpeB = weights_recordB.iloc[max_sharpe_idx]

In [26]:
# Calculate daily returns for the maximum Sharpe portfolio using only the original strategies
max_sharpe_daily_returnsB = (myStrategies_dfB * weights_max_sharpeB).sum(axis=1)

### Out Sample Code:

#### Key Steps:
- **Applying MVO Weights**: The portfolio weights derived from the Maximum Sharpe Ratio calculation in Part A were re-applied to the chosen instruments' returns over the full period. This resulted in the creation of the MVO-weighted portfolio for the in-sample and out-of-sample periods.
  
- **Performance Evaluation**: The performance statistics (including Sharpe ratios) for the MVO-weighted portfolio were computed for both sub-periods. The **in-sample Sharpe ratio** was verified to be identical to the one calculated in Part A (Question 16), confirming consistency in our method.
  
- **Out-of-Sample Sharpe Ratio**: The Sharpe ratio for the new out-of-sample period was highlighted to assess how well the portfolio performed outside of the initial training period. This comparison allowed us to determine whether the strategies continued to offer favorable risk-adjusted returns after 2018.

In [27]:
# Run the function on all 12 instruments and store the results
for instrument, params in strategy_params.items():
    strategy_type, fastWindow, slowWindow, bbWindow, stdevBand = params
    # Use the previously defined runMovingAverageAndBB function
    resultB_df_outSample = runMovingAverageAndBB(pricesB_df_outOfSample[[instrument]], fastWindow, slowWindow, bbWindow, stdevBand)
    
    # Append the result to the list of dataframes
    dataframesB_outSample.append(resultB_df_outSample)

In [28]:
# Concatenate all 12 dataFrames horizontally to create a master DataFrame with 60 columns
masterB_df_outSample = pd.concat(dataframesB_outSample, axis=1)

# Display the master DataFrame
#masterB_df.head()


In [29]:
#calcPerformanceStatistics(myStrategies_dfB_outSample_cleaned)

In [30]:
# Sort the strategies alphabetically
myStrategies_sorted = sorted(myStrategies)


In [31]:
# Subset the master_df to include only the columns for the chosen strategies
myStrategies_dfB_outSample = masterB_df_outSample[myStrategies_sorted]

In [32]:
myStrategies_dfB_outSample_cleaned = remove_duplicate_columns(myStrategies_dfB_outSample)

In [33]:
np.random.seed(64)  # For reproducibility

# returns = myStrategies_df.pct_change().dropna()  # Daily returns of the instruments
annual_returns = myStrategies_dfB_outSample_cleaned.mean() * 260  # Annualized returns
cov_matrix = myStrategies_dfB_outSample_cleaned.cov() * 260  # Annualized covariance matrix

num_portfolios = 5000  # Monte Carlo simulation
resultsB_outSample = np.zeros((3, num_portfolios))
weights_recordB = []

for i in range(num_portfolios):
    weights = np.random.random(len(annual_returns))
    weights /= np.sum(weights)  # Normalize weights to sum to 1
    weights_recordB.append(weights)

    # Portfolio return and volatility
    portfolio_return = np.sum(weights * annual_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Sharpe ratio assuming risk-free rate = 0
    sharpe_ratio = portfolio_return / portfolio_volatility

    resultsB_outSample[0, i] = portfolio_return
    resultsB_outSample[1, i] = portfolio_volatility
    resultsB_outSample[2, i] = sharpe_ratio

weights_recordB = pd.DataFrame(weights_recordB, columns=myStrategies_dfB_outSample_cleaned.columns)

In [34]:
max_sharpe_idx = np.argmax(resultsB_outSample[2, :])


In [36]:
# Calculate daily returns for the maximum Sharpe portfolio using only the original strategies
max_sharpe_daily_returnsB_outSample = (myStrategies_dfB_outSample_cleaned * weights_max_sharpeB).sum(axis=1)

In [37]:
# Create a copy of myStrategies_df before modifying
myStrategies_dfB_outSample_cleaned = myStrategies_dfB_outSample_cleaned.copy()


In [38]:
# Calculate the performance statistics for the updated DataFrame
performance_statistics_B = calcPerformanceStatistics(myStrategies_dfB_outSample_cleaned)

# Display the performance statistics
print(performance_statistics_B)

                 AAPL-MAFlat  AMZN-MAFlat    AUD-BB  FBNDX-BMK-MA  GE-MAShort  \
Annual Return       0.241749     0.142483 -0.024656     -0.063749   -0.404041   
Annualized Risk     0.355839     0.293982  0.062498      0.059039    0.475000   
Sharpe Ratio        0.679377     0.484666 -0.394507     -1.079785   -0.850613   
Max Drawdown       -0.377275    -0.295447 -0.111188     -0.264283   -1.485787   
Success Ratio       0.583807     0.661932  0.869318      0.578125    0.473011   

                 GOLD-MAShort   INTC-BB  SPY-MAFlat  
Annual Return        0.028694  0.053512    0.009703  
Annualized Risk      0.164776  0.245252    0.230624  
Sharpe Ratio         0.174137  0.218191    0.042073  
Max Drawdown        -0.276436 -0.274141   -0.417104  
Success Ratio        0.529830  0.911932    0.656250  


### In Sample Code

In [39]:
# Run the function on all 12 instruments and store the results
for instrument, params in strategy_params.items():
    strategy_type, fastWindow, slowWindow, bbWindow, stdevBand = params
    # Use the previously defined runMovingAverageAndBB function
    resultB_df_inSample = runMovingAverageAndBB(pricesB_df_inSample[[instrument]], fastWindow, slowWindow, bbWindow, stdevBand)
    
    # Append the result to the list of dataframes
    dataframesB_inSample.append(resultB_df_inSample)

In [40]:
# Concatenate all 12 dataFrames horizontally to create a master DataFrame with 60 columns
masterB_df_inSample = pd.concat(dataframesB_inSample, axis=1)


In [41]:
# Sort the strategies alphabetically
myStrategies_sorted = sorted(myStrategies)


In [42]:
# Subset the master_df to include only the columns for the chosen strategies
myStrategies_dfB_inSample = masterB_df_inSample[myStrategies_sorted]

In [43]:
myStrategies_dfB_inSample_cleaned = remove_duplicate_columns(myStrategies_dfB_inSample)

In [44]:
np.random.seed(64)  # For reproducibility

# returns = myStrategies_df.pct_change().dropna()  # Daily returns of the instruments
annual_returns1 = myStrategies_dfB_inSample_cleaned.mean() * 260  # Annualized returns
cov_matrix1 = myStrategies_dfB_inSample_cleaned.cov() * 260  # Annualized covariance matrix

num_portfolios = 5000  # Monte Carlo simulation
resultsB_inSample = np.zeros((3, num_portfolios))
weights_recordB1 = []

for i in range(num_portfolios):
    weights = np.random.random(len(annual_returns1))
    weights /= np.sum(weights)  # Normalize weights to sum to 1
    weights_recordB1.append(weights)

    # Portfolio return and volatility
    portfolio_return = np.sum(weights * annual_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Sharpe ratio assuming risk-free rate = 0
    sharpe_ratio = portfolio_return / portfolio_volatility

    resultsB_inSample[0, i] = portfolio_return
    resultsB_inSample[1, i] = portfolio_volatility
    resultsB_inSample[2, i] = sharpe_ratio

weights_recordB = pd.DataFrame(weights_recordB1, columns=myStrategies_dfB_inSample_cleaned.columns)

In [45]:
max_sharpe_idx = np.argmax(resultsB_inSample[2, :])


In [46]:
# Calculate daily returns for the maximum Sharpe portfolio using only the original strategies
max_sharpe_daily_returnsB_inSample = (myStrategies_dfB_inSample_cleaned * weights_max_sharpeB).sum(axis=1)

In [47]:
# Create a copy of myStrategies_df before modifying
myStrategies_dfB_inSample_cleaned = myStrategies_dfB_inSample_cleaned.copy()

In [48]:
# Calculate the performance statistics for the updated DataFrame
performance_statistics_B = calcPerformanceStatistics(myStrategies_dfB_inSample_cleaned)

# Display the performance statistics
print(performance_statistics_B)

                 AAPL-MAFlat  AMZN-MAFlat    AUD-BB  FBNDX-BMK-MA  GE-MAShort  \
Annual Return       0.222849     0.260893 -0.000278      0.003581    0.128027   
Annualized Risk     0.296838     0.330598  0.057321      0.038143    0.308737   
Sharpe Ratio        0.750743     0.789155 -0.004843      0.093893    0.414681   
Max Drawdown       -0.704743    -0.479291 -0.183208     -0.236986   -1.510200   
Success Ratio       0.655457     0.666147  0.968151      0.646102    0.523608   

                 GOLD-MAShort   INTC-BB  SPY-MAFlat  
Annual Return        0.071038 -0.031812    0.056340  
Annualized Risk      0.179685  0.113463    0.117781  
Sharpe Ratio         0.395347 -0.280369    0.478349  
Max Drawdown        -0.449747 -0.721082   -0.229252  
Success Ratio        0.532071  0.946325    0.687751  


#### Results:
- The in-sample Sharpe ratio matched the Part A results, confirming the robustness of the MVO optimization approach.
- The out-of-sample Sharpe ratio provided new insights, helping us evaluate the strategy's continued effectiveness during the recent period, which included events like the COVID-19 market volatility.

#### Conclusion:
This analysis showed whether the MVO Maximum Sharpe portfolio's out-of-sample performance was better or worse than the in-sample performance, helping us determine the longevity of our optimized portfolio.

### Question 5: Compare Active Portfolio's Out-of-Sample Sharpe with Passive Benchmark Portfolio

#### Reasoning:
In this question, we aimed to evaluate the performance of our active portfolio compared to a passive equal-weighted portfolio that only includes the benchmarks (i.e., the "-BMK-MA" versions of the instruments). The goal is to analyze whether our actively managed strategies outperform a simple, passive strategy that does not involve any technical trading strategies.

#### Key Steps:
1. **Constructing the Passive Portfolio**:
   - We created an equal-weighted portfolio composed of the benchmark versions ("-BMK-MA") of the chosen instruments. This portfolio does not include any technical strategies like MAFlat or Bollinger Bands and simply holds the benchmark instruments over time.
   
2. **Out-of-Sample Period**:
   - The performance statistics of this passive portfolio were calculated for the out-of-sample period from **01/02/2019 to 10/31/2022**. The focus was on the Sharpe ratio to assess the risk-adjusted return of the passive portfolio during this period.
   
3. **Comparison with Active Portfolio**:
   - The passive portfolio's Sharpe ratio was then compared to the Sharpe ratio of the active portfolio over the same out-of-sample period. This allowed us to see whether the active strategies we employed delivered better risk-adjusted returns than simply holding a passive benchmark portfolio.

In [49]:
# Extract BMK-only portfolio returns from master_df for the out-of-sample period
bmk_columns = [f'{instrument}-BMK-MA' for instrument in selected_instruments]
bmk_only_returns_out_of_sample = masterB_df.loc['2019-01-02':'2022-10-31', bmk_columns].mean(axis=1)  # Equal-weighted portfolio returns

# Calculate the Sharpe ratio for the BMK-only portfolio during the out-of-sample period
bmk_sharpe_out_of_sample = calculate_portfolio_sharpe(bmk_only_returns_out_of_sample)

# Compare with Max Sharpe Portfolio for the out-of-sample period
max_sharpe_returns_out_of_sample = max_sharpe_daily_returnsB.loc['2019-01-02':'2022-10-31']  # Subset to out-of-sample period
max_sharpe_ratio_out_of_sample = calculate_portfolio_sharpe(max_sharpe_returns_out_of_sample)

# Display the Sharpe ratios for the out-of-sample period
print(f"Sharpe Ratio for BMK-only Portfolio (Out-of-Sample): {bmk_sharpe_out_of_sample:.4f}")
print(f"Sharpe Ratio for Max Sharpe Portfolio (Out-of-Sample): {max_sharpe_ratio_out_of_sample:.4f}")

# Check if BMK-only portfolio outperforms the Max Sharpe portfolio during the out-of-sample period
if bmk_sharpe_out_of_sample > max_sharpe_ratio_out_of_sample:
    print("BMK-only Portfolio outperforms the Max Sharpe Portfolio in the out-of-sample period.")
else:
    print("BMK-only Portfolio does not outperform the Max Sharpe Portfolio in the out-of-sample period.")

# Calculate mean return for the out-of-sample period
mean_return = bmk_only_returns_out_of_sample.mean()

# Calculate volatility (standard deviation) for the out-of-sample period
volatility = bmk_only_returns_out_of_sample.std()

# Calculate the Sharpe ratio (already computed)
sharpe_ratio = bmk_sharpe_out_of_sample

# Calculate annualized return (assuming 260 trading days in a year)
annualized_return = mean_return * 260

# Calculate annualized volatility (assuming 260 trading days in a year)
annualized_volatility = volatility * np.sqrt(260)

# Optional: Calculate maximum drawdown
cumulative_returns = (1 + bmk_only_returns_out_of_sample).cumprod()
rolling_max = cumulative_returns.cummax()
drawdown = (cumulative_returns - rolling_max) / rolling_max
max_drawdown = drawdown.min()

# Print summary statistics
print(f"Summary Statistics for BMK-only Portfolio (Out-of-Sample Period):")
print(f"Mean Return (Daily): {mean_return:.4f}")
print(f"Volatility (Daily): {volatility:.4f}")
print(f"Annualized Return: {annualized_return:.4f}")
print(f"Annualized Volatility: {annualized_volatility:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Maximum Drawdown: {max_drawdown:.4f}")

Sharpe Ratio for BMK-only Portfolio (Out-of-Sample): 0.3528
Sharpe Ratio for Max Sharpe Portfolio (Out-of-Sample): 0.6863
BMK-only Portfolio does not outperform the Max Sharpe Portfolio in the out-of-sample period.
Summary Statistics for BMK-only Portfolio (Out-of-Sample Period):
Mean Return (Daily): 0.0003
Volatility (Daily): 0.0115
Annualized Return: 0.0655
Annualized Volatility: 0.1855
Sharpe Ratio: 0.3528
Maximum Drawdown: -0.2877


#### Results:
- **Passive Portfolio Sharpe Ratio**: The passive portfolio’s out-of-sample Sharpe ratio was highlighted to assess its performance relative to the active portfolio. 
- **Active vs. Passive**: If the active portfolio’s Sharpe ratio is higher, it demonstrates that active management and the use of technical strategies added value. If the passive portfolio’s Sharpe ratio is higher, it suggests that a simple buy-and-hold strategy might be more effective for this period.

#### Conclusion:
This comparison provided insights into whether active management outperformed the passive benchmark in terms of risk-adjusted returns during the out-of-sample period. It helps to assess the overall effectiveness and viability of our chosen active strategies versus a simple passive approach.

### Question 6: Comment on Active vs. Passive Portfolio Performance

#### Reasoning:

In this final comparison, we evaluated whether the active portfolio outperformed the passive benchmark over the out-of-sample period. While the passive portfolio reflects market performance, the active portfolio's success depends on its ability to consistently outperform through strategic allocations and market timing. The conclusion drawn from this comparison helps us assess the value of active management in our portfolio. If the active portfolio consistently provides better risk-adjusted returns, it justifies the additional effort and complexity involved in active management.